# Train HOG

In [110]:
# Importing dependencies

from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *

In [111]:
# define parameters of HOG feature extraction
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

In [112]:
# define path to images:

pos_im_path = r"../../datasets/kaggle_human01/human" # This is the path of our positive input dataset
# define the same for negatives
neg_im_path= r"../../datasets/kaggle_human01/no_human"

In [113]:
# read the image files:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)
num_pos_samples = size(pos_im_listing) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)
print(num_pos_samples) # prints the number value of the no.of samples in positive dataset
print(num_neg_samples)
data= []
labels = []

559
362


In [114]:
# compute HOG features and label them:

for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    img = Image.open(pos_im_path + '/' + file) # open the file
    img = img.resize((256, 256))
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    data.append(fd)
    labels.append(1)

type(fd)

numpy.ndarray

In [115]:
# Same for the negative images
for file in neg_im_listing:
    img= Image.open(neg_im_path + '/' + file)
    img = img.resize((256, 256))
    gray= img.convert('L')
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True) 
    data.append(fd)
    labels.append(0)
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

In [116]:
# Partitioning the data into training and testing splits, using 80%
# of the data for training and the remaining 20% for testing
np.stack(data, axis=0)
print(shape(data))
print(type(labels))

print(" Constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data, dtype=object), labels, test_size=0.20, random_state=42)

print(shape(trainData))
print(shape(trainLabels))

# %% Train the linear SVM
print(" Training Linear SVM classifier...")
model = LinearSVC()
model.fit(trainData, trainLabels)
#%% Evaluate the classifier
print(" Evaluating classifier on test data ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

(921, 34596)
<class 'numpy.ndarray'>
 Constructing training/testing split...
(736, 34596)
(736,)
 Training Linear SVM classifier...
 Evaluating classifier on test data ...
              precision    recall  f1-score   support

           0       0.60      0.51      0.55        72
           1       0.72      0.78      0.75       113

    accuracy                           0.68       185
   macro avg       0.66      0.65      0.65       185
weighted avg       0.67      0.68      0.67       185



/home/drazen/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [117]:
# Save the model:
#%% Save the Model
joblib.dump(model, 'hog_model.npy')

['hog_model.npy']